# CART (Classification and Regression Trees)
## Introduction 
Decision trees are a simple yet remarkably powerful models which can be used in both classification and regression problems. Decision trees categorize data by proposing a question based on each feature (or a subset of features) from your dataset, and then splitting the data based on that question. There are many different types of decision trees, but they all work on the same premise: 

**All decision trees partition the data recursively by examining the features of the data, and choosing the feature(s) that best split the data. In other words, these decisions are based on maximizing data homogenity (similarity) within the trees leaf nodes, and heterogenity (difference) between the leaf nodes.**

Since such models are built on the premise of the labeled data we provide them, they belong to the *supervised learning* family of algorithms. This notebook examines the CART algorithm, a common decision tree algorithm which produces binary classification or regression trees based on wether the data is categorical or numeric. 

# CART for Classification
Below is an example of a very simple decision tree for classification. We use this tree as an example to describe how CART works in classification problems. 
<br>
<br>

<img src="images_and_data/tree.png" width=400 height=400 />

A CART decision tree repeatedly splits the data at each **node** by proposing a question with a binary outcome. The first node is the root node, and the last nodes are the leaf nodes which store our predictions. For our simple made up dataset, it was easy to construct a tree that classified our samples with 100% accuracy. We only have three entries and very few features. However, when handling datasets which are much larger (think thousands of different fruits with many more features) the ideal decision tree structure becomes far less apparent. Constructing a good decision tree involves understanding *what questions to ask, and why to ask them.* Ideally, we want to choose our questions in a way that best classifies our data, and works well at predicting any new data. We would like a rigorous theoretical framework to arrive at the best possible tree structure. CART uses two key concepts to achieve this:

1. A measure of *impurity* with the **Gini score**
2. A measure of **information gain**

## Gini Impurity
The Gini impurity describes the **probability that a label is incorrectly assigned to a randomly chosen example in a set of data.** We pay particular attention to the **difference in Gini score between two sets of data** when creating a partition (we elaborate on this later on.) First, we describe the formula for the Gini impurity.

### Gini impurity = $ \sum_{n=0}^{i} p_{i}~(1~-~p_{i})$
where $p_{i}$ is the probability of the label 'i' being chosen. We can simplify this formula:
### $ \sum_{n=0}^{i} p_{i}~(1~-~p_{i}) $ = $ \sum_{n=0}^{i} p_{i}~-\sum_{n=0}^{i}~(p_{i})^2 $ 
### $ \sum_{n=0}^{i} p_{i} $ = 1
Therefore 
### Gini impurity = $ 1 - \sum_{n=0}^{i} (p_{i})^2 $

<br>

Passing our data through our example decision tree results in our leaf nodes having zero impurity (0 gini score.) This is because for our three examples, all our leaf nodes end up having exactly one type of fruit (it is impossible to mismatch the one existing label at each node.) The outcome is considered perfectly pure. On the other hand, if we instead introduced another fruit, say a yellow pineapple with height 8cm, then our leftmost leafnode would classify pineapple and banana in the same node. This results in an impurity of 50% (i.e a 50% chance that we classify incorrectly.) 

Our goal is to choose our questions at each node to *minimize the gini impurity.* Below is a step by step description of how this is done. 

1. Compute the impurity of the starting set of data
2. Ask a question based on a feature in the data. 
3. Compute the weighted average gini impurity from the resulting leaf nodes of that question
4. Compute the *difference in Gini scores from before the question was asked, and the weighted average Gini impurity in step 3.*
5. Repeat steps 2 through 4 for all the features , and choose the question which produces the **biggest drop in Gini impurity**. 

Note, that if the best reduction in Gini impurity we can obtain is zero, we no longer split the node and it becomes a leaf node. 

Using these steps, we recursively build each branch of the tree until there are no more questions left to ask. We summarize these steps below using our original made up fruit data table, but slightly modified to have a quantity feature:

<table><tr>
<td> <img src="images_and_data/step_1.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="images_and_data/step_2.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>

<table><tr>
<td> <img src="images_and_data/step_3.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="images_and_data/step_4.png" alt="Drawing" style="width: 350px;"/> </td>
</tr></table>

<br>

<img src="images_and_data/step_5.png" width=300 height=300 />


# CART for Regression 
CART can be used in cases where the target variable is continuous. In such a case, CART assumes a regression framework and needs a measure of how well its predictions model the original data. Just like linear regression, CART takes advantage of the **Least Squares Deviation**, where the goal is to minimize the sum of the squared residuals between the predicted values and the datapoints.

## Demonstrating Least Squares using 1 Predictor 
To demonstrate, we use a made up graph of some fictional populations monthly rent as a function of disposable income. In this case, we are using 1 predictor (income) to try and model montly rent. 1 predictor also makes it easier to demonstrate the least squares regression. 

<img src="images_and_data/data.png" width=400 height=400 />
<br>
CART begins its regression by iterating through each datapoint, taking the predictor value and carrying out these steps:  

1. Produce a partition based based on the datapoint. 
2. Compute the residual sum of squares for both children nodes. For each child node, we comute the residual with respect to the average value of the data that falls into each node. 
3. **Choose the the datapoint which produces the smallest residual sum of squares (RSS) as the partition value at the node.**
4. Recursively reapply the same steps to the chidren nodes and build the tree until some stopping criteria is reached

Stopping criteria is elaborated on later on.  
<table><tr>
<td> <img src="images_and_data/reg_s1.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="images_and_data/reg_partition.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="images_and_data/reg_s2.png" alt="Drawing" style="width: 500px;"/> </td>
</tr></table>
<img src="images_and_data/reg_s34.png" width=350 height=400 />
<br>
Though we only use one predictor in this example, CART can extend this least squares approach to multiple predictors. 

# Evaluating a CART Model
Evaluating a CART DT follows a similar approach to to many other modelling types. We split out data into training  and test sets. The DT is fed a training set to construct the model, and various hyperparameters (such as DT depth, or other custom stopping criteria like minimum number of datapoints per leaf node) can be tuned.


# CART classification using scikit-learn
We write a simple implementation of a CART DT using scikit learn. For our data, we borrow a diabetes dataset which lists a number of patients, some diagnostic data (blood pressure, glucose, BMI) and finally a diabetes diagnosis (yes/no). We aim to construct a CART DT which best predicts the whether a patient has diabetes absed on this diagnostic information. 

In [ ]:
import warnings
import pandas as pd

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from sklearn.tree import DecisionTreeClassifier # DT Classifier
from sklearn import metrics # Used to compute accuracy of model
from sklearn.model_selection import train_test_split # Used to split data into train and test sets
 
# Extract data
diabetes_data = pd.read_csv("/Users/srujanvajram/Documents/Github/ML_playground/ML_playground/decision_trees/CART/images_and_data/diabetes.csv")

# Grab the predictor variables and target variable separately 
predictors = diabetes_data.iloc[:, :-1]
targets = diabetes_data.iloc[:, -1]

# Split data into test and training sets 
# 80/20 split among training : test
predictor_train, predictor_test, target_train, target_test = train_test_split(predictors, 
                                                                              targets, 
                                                                              test_size=0.2, 
                                                                              random_state=1)


# Instantiate classifier
CART_classifier = DecisionTreeClassifier(criterion = 'gini')

# Training phase
CART_classifier = CART_classifier.fit(predictor_train,target_train)

# Predict the response for test dataset
model_prediction = CART_classifier.predict(predictor_test)

# Print the training accuracy 
print("Model accuracy:", metrics.accuracy_score(target_test, model_prediction))

## Decision tree weaknesses 
Where CART excells in its interpretability and ease of implementation, it often performs poorly when trying to **generalize to new data.** Given how CART discriminates purely based on the data features, a single tree if allowed to grow uninterrupted to any depth will often assume high variance (i.e overfit the data.) This issue is in fact not specific to CART, but decision tree models in general. 

Fortunately there are a number of methods at our disposal to combat these issues. These usually involve reducing the tree structure complexity using **pruning methods.** Despite pruning, a single decision tree typically will not perform very well at classification and regression tasks. Fortunately, producing multiple decision trees and taking a consensus from them has proven a far more succesful modelling strategy. These multiple tree strategies are broadly classified as **ensemble methods.** We touch upon different ensemble methods in a separate notebook.

## Improving accuracy by reducing variance
With our current model, we are getting close to a 70% accuracy. We suspect there is some overfitting happening which is leading to poor generalization to the test set. As such, we can employ a pruning technique to reduce the complexity of the decision boundaries drawn by our model. One technique is **limiting tree depth.** By restricting tree depth, we are implicitly reducing the number of decision boundaries drawn by the tree, and hopefully improving the models ability to generalize to new data. 

We can pass tree depth directly as an argument to the DecisionTreeClassifier

In [134]:
# Instantiate classifier
CART_classifier = DecisionTreeClassifier(criterion = 'gini', max_depth = 2)

# Redo training and predictions
CART_classifier = CART_classifier.fit(predictor_train,target_train)
model_prediction = CART_classifier.predict(predictor_test)

# Print the training accuracy 
print("Model accuracy:", metrics.accuracy_score(target_test, model_prediction))

Model accuracy: 0.7987012987012987


Tree depth as such becomes a hyperparamter that can be tuned. Note that tree depth is one of a few pruning techniques offered by scikit-learn.  *Cost complexity (ccp_alfa)* is another common post pruning technique where a fully grown tree is reduced to subtrees by removing nodes to reduce the error rate. 
